# Create tweet tables in your sql database

## Filter for time and geometry

In [ ]:
years= [2020,2021,2022]
for year in years:

    tweets_us_table= f"""create table covid.tweets_{str(year)} TABLESPACE extern_hard_disc as (
    SELECT message_id, date, text, place, geom, retweets, user_id, followers, latitude, longitude 
        FROM archive.twitter 
        WHERE date > '{str(year-1)}-12-31'
        AND date < '{str(year+1)}-01-01'
        AND  ST_Contains(ST_Geomfromtext('POLYGON((-94.29626464843747 48.82253520412155,-89.33166503906247 48.086482939580165,-88.40454101562496 48.49930883702451,-81.82617187499996 45.556997304202724,-82.12738037109368 43.58413037683304,-82.40661621093744 42.019577581169486,-78.82598876953122 43.62158420926344,-75.15747070312496 45.10452321662082,-72.00256347656244 45.19416468029458,-69.63409423828121 47.58653720888464,-67.26562499999997 47.3621620334887,-66.42333984375 44.98354963777504,-73.8427734375 33.684946574921526,-76.19628906250003 26.48320357120556,-80.57128906250001 23.63324079406921,-97.66601562499999 25.36728759218286,-103.24707031250001 28.939007844711835,-116.58447265625003 31.32803288903339,-124.64843750000001 33.584746186613586,-128.35937499999997 49.243899719809306,-95.37017822265624 49.06606565614926,-95.24520874023436 49.4362797026685,-94.79820251464842 49.42779507018419,-94.60765838623045 48.956849452333515,-94.29626464843747 48.82253520412155))',4326), ST_Centroid(geom))
    ) """
    print(tweets_us_table)
    #run_querry()

## Filter for keywords

In [ ]:
topics_dict = {
    "virus": "covid|corona|sarscov|sars-cov|epidemic|pandemic|influenza|virus|viral|infect|2019-ncov|Delta variant|Omicron|H1N1|H3N2|Wuhan|transmission|super spread|incubation",
    "symptoms": "fever|cough|shortness of breath|sore throat|headache|fatigue|body aches|loss of taste|loss of smell|no smell|no taste|nasal congestion|runny nose|respirator|symptom",
    "testing": "PCR|antigen|rapid|test",
    "vaccination": "vaccin|booster|Pfizer|Moderna|AstraZeneca|Johnson & Johnson|comirnaty|Janssen|mrna|vax| jab |biontech",
    "preventive_measures": "face mask|face covering|FFP2|N95|KN95|KF94|staysafe|stay safe|flattenthecurve|flatten the curve|handwashing|wash your hands|mask mandat|mask",
    "quarantine": "quarantine|lockdown|social distancing|stay-at-home|stay at home|stayhome|stayathome|stay-home|stay home|isolat|socialdistance|social distancing|keepdistance|keep distance",
    "health_experts_cons": "health expert|Fauci|world health organization|cdc|centers for disease control|virologist|immunologist|supportive care|hospital|ventilat|clinic|Intensive Care Unit|FDA",
    "covid_baseline_topic": "covid|corona|sarscov|sars-cov|epidemic|pandemic|influenza|virus|viral|infect|2019-ncov|Delta variant|Omicron|H1N1|H3N2|Wuhan|transmission|super spread|incubation|fever|cough|shortness of breath|sore throat|headache|fatigue|body aches|loss of taste|loss of smell|no smell|no taste|nasal congestion|runny nose|respirator|symptom|PCR|antigen|rapid|test|vaccin|booster|Pfizer|Moderna|AstraZeneca|Johnson & Johnson|Comirnaty|Janssen|mrna|vax|biontech| jab |FDA|face mask|face covering|FFP2|N95|KN95|KF94|staysafe|stay safe|flattenthecurve|flatten the curve|handwashing|wash your hands|mask mandat|mask|quarantine|lockdown|social distancing|stay-at-home|stay at home|stayhome|stayathome|stay-home|stay home|isolat|socialdistance|social distancing|keepdistance|keep distance|health expert|Fauci|world health organization|cdc|centers for disease control|virologist|immunologist|supportive care|hospital|ventilat|clinic|Intensive Care Unit"
}

In [ ]:

# Collect all keywords from prior topics
prior_keywords = set()
for key, value in topics_dict.items():
    if key != "covid_baseline_topic":
        prior_keywords.update(value.split("|"))

# Collect all keywords from the baseline topic
baseline_keywords = set(topics_dict["covid_baseline_topic"].split("|"))

# Find missing keywords
missing_keywords = prior_keywords - baseline_keywords

# Print missing keywords
if missing_keywords:
    print("Missing keywords in covid_baseline_topic:")
    print(missing_keywords)
else:
    print("All keywords are present in covid_baseline_topic.")


In [ ]:
for year in years:
    for topic in topics_dict.keys():
        filter_for_keywords=f"""ALTER TABLE covid.tweets_{str(year)} ADD COLUMN {topic} bigint default 0;
        UPDATE covid.tweets_{str(year)}
        SET {topic} = 1
        WHERE lower(text) similar to lower('%({topics_dict[topic]})%');"""
        print(filter_for_keywords)
        

## Add geometries

In [ ]:
for year in years:

    add_geom_id = f"""ALTER TABLE
    covid.tweets_{str(year)}
    ADD COLUMN state_id int;"""
    print(add_geom_id)

    create_geom_idx=f"""
    create index idx_spatial_{str(year)} on covid.tweets_{str(year)} using GIST(geom)
    """
    print(create_geom_idx)

    match_geom_ids=f"""
    UPDATE covid.tweets_{str(year)} tw
    SET state_id = ol.state_id
    from covid.states as ol
    WHERE
    st_within (tw.geom, ol.geometry);
    """
    print(match_geom_ids)
    break

## Groupby and match per date and geoid

In [ ]:
for year in years:
    create_aggregation_table=f"""create table covid.tweets_{str(year)}_agg_state_cons as (
                                    select (to_char(date, 'YYYY-MM-DD'::text)) as date, state_id, count(*) as all_tweets,
                                    sum({list(topics_dict.keys())[0]}) as {list(topics_dict.keys())[0]}
                                    from covid.tweets_{str(year)}
                                    GROUP BY (to_char(date, 'YYYY-MM-DD'::text)), state_id )"""
    print(create_aggregation_table)
    for topic in list(topics_dict.keys())[1:]:

        alter_aggr_table=f"""ALTER TABLE covid.tweets_{str(year)}_agg_state ADD COLUMN {topic} bigint default 0;"""
        print(alter_aggr_table)

        update_aggr_table=f"""UPDATE covid.tweets_{str(year)}_agg_state AS target 
                                SET {topic} = source.{topic}
                                FROM (
                                select (to_char(date, 'YYYY-MM-DD'::text)) as date, state_id, count(*) as all_tweets,
                                sum({topic}) as {topic}
                                from covid.tweets_{str(year)}
                                GROUP BY (to_char(date, 'YYYY-MM-DD'::text)), state_id
                                ) AS source
                                WHERE target.date = source.date
                                AND target.state_id = source.state_id; """
        print(update_aggr_table)
        break
    break

